In [ ]:
#!pip install rdkit-pypi
#!pip install tensorflow_datasets
#!pip install --upgrade tensorflow

In [5]:
#!pip install -e ../.

In [8]:
import rdkit


In [14]:
from feature_extractor import fingerprint_features

In [ ]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [31]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [32]:
import numpy as np
def get_fingerprint(smiles):
    return np.frombuffer(fingerprint_features(smiles).ToBitString().encode(), 'u1') - ord('0')

In [22]:
import pandas as pd
data = pd.read_csv("../data/0_raw/data.csv").drop("mol_id", axis=1)
#data["fingerprint"] = data["smiles"].apply(get_fingerprint)

In [34]:
X = data["smiles"].values
y = data["P1"].values

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

In [37]:
len(X_train_fingerprint), len(X_test_fingerprint)

(4249, 750)

In [38]:
train_features = np.float_(X_train_fingerprint)
train_labels = y_train
test_features = np.float_(X_test_fingerprint)
test_labels = y_test


In [39]:
len(train_features), len(y_train)

(4249, 4249)

In [46]:
import tensorflow as tf
model = tf.keras.Sequential([
  tf.keras.layers.Dense(56, input_shape=(2048,)),
  tf.keras.layers.Dropout(0.6),
  tf.keras.layers.Dense(56, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(1,activation="sigmoid"),
  ])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [50]:
inputs = keras.Input(shape=(2048,))
x = keras.layers.Dense(30, activation="relu")(inputs)
outputs = keras.layers.Dense(1, activation='sigmoid',)(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="servier_model")
model.summary()

Model: "servier_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 2048)]            0         
                                                                 
 dense_21 (Dense)            (None, 30)                61470     
                                                                 
 dense_22 (Dense)            (None, 1)                 31        
                                                                 
Total params: 61,501
Trainable params: 61,501
Non-trainable params: 0
_________________________________________________________________


In [61]:
from tensorflow import keras
METRICS = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    #inputs = keras.Input(shape=(2048,))
    #keras.layers.Dense(30, activation="relu")
    model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(2048,)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
    ])

    model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

    return model

In [62]:
EPOCHS = 100
BATCH_SIZE = 128

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [63]:

def make_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    inputs = keras.Input(shape=(2048,))
    x = keras.layers.Dense(30, activation="relu")(inputs)
    outputs = keras.layers.Dense(1, activation='sigmoid',)(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name="servier_model")
    
    model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

    return model

In [64]:
model = make_model()
model.summary()

Model: "servier_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 2048)]            0         
                                                                 
 dense_33 (Dense)            (None, 30)                61470     
                                                                 
 dense_34 (Dense)            (None, 1)                 31        
                                                                 
Total params: 61,501
Trainable params: 61,501
Non-trainable params: 0
_________________________________________________________________


In [65]:
history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_split=0.2,
    #validation_data=(test_features, test_labels),
    )

Epoch 1/100


2022-04-14 13:41:23.090516: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - ETA: 0s - loss: 0.5724 - accuracy: 0.7132 - precision: 0.8168 - recall: 0.8387 - auc: 0.4956 - prc: 0.8233   

2022-04-14 13:41:23.992945: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - 2s 38ms/step - loss: 0.5724 - accuracy: 0.7132 - precision: 0.8168 - recall: 0.8387 - auc: 0.4956 - prc: 0.8233 - val_loss: 0.4616 - val_accuracy: 0.8259 - val_precision: 0.8259 - val_recall: 1.0000 - val_auc: 0.5472 - val_prc: 0.8638
Epoch 2/100
27/27 [==============================] - 0s 18ms/step - loss: 0.4539 - accuracy: 0.8208 - precision: 0.8208 - recall: 1.0000 - auc: 0.6356 - prc: 0.8878 - val_loss: 0.4470 - val_accuracy: 0.8259 - val_precision: 0.8259 - val_recall: 1.0000 - val_auc: 0.6320 - val_prc: 0.8888
Epoch 3/100
27/27 [==============================] - 0s 18ms/step - loss: 0.4244 - accuracy: 0.8208 - precision: 0.8208 - recall: 1.0000 - auc: 0.7507 - prc: 0.9266 - val_loss: 0.4379 - val_accuracy: 0.8259 - val_precision: 0.8259 - val_recall: 1.0000 - val_auc: 0.6622 - val_prc: 0.8989
Epoch 4/100
27/27 [==============================] - 0s 18ms/step - loss: 0.4015 - accuracy: 0.8220 - precision: 0.8220 - recall: 0.9996 - auc: 0.79

In [65]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
total = neg + pos
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 2.81
Weight for class 1: 0.61


In [67]:
weighted_model = make_model()
weighted_model.load_weights(initial_weights)

weighted_history = weighted_model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(test_features, test_labels),
    # The class weights go here
    class_weight=class_weight)

Epoch 1/100
3/3 [==============================] - 2s 427ms/step - loss: 0.7374 - tp: 3871.0000 - fp: 803.0000 - tn: 88.0000 - fn: 237.0000 - accuracy: 0.7920 - precision: 0.8282 - recall: 0.9423 - auc: 0.5130 - prc: 0.8298 - val_loss: 0.5939 - val_tp: 607.0000 - val_fp: 132.0000 - val_tn: 2.0000 - val_fn: 9.0000 - val_accuracy: 0.8120 - val_precision: 0.8214 - val_recall: 0.9854 - val_auc: 0.5287 - val_prc: 0.8376
Epoch 2/100
3/3 [==============================] - 0s 29ms/step - loss: 0.6991 - tp: 3151.0000 - fp: 658.0000 - tn: 99.0000 - fn: 341.0000 - accuracy: 0.7649 - precision: 0.8273 - recall: 0.9023 - auc: 0.5649 - prc: 0.8553 - val_loss: 0.6220 - val_tp: 566.0000 - val_fp: 121.0000 - val_tn: 13.0000 - val_fn: 50.0000 - val_accuracy: 0.7720 - val_precision: 0.8239 - val_recall: 0.9188 - val_auc: 0.5658 - val_prc: 0.8565
Epoch 3/100
3/3 [==============================] - 0s 29ms/step - loss: 0.6882 - tp: 2810.0000 - fp: 544.0000 - tn: 213.0000 - fn: 682.0000 - accuracy: 0.7115 - 